정수 리터럴과 덧셈만 있는 아주 간단한 산술식을 하스켈으로 다뤄보자.

In [1]:
:opt no-lint                          -- linter 끄기
{-# LANGUAGE ScopedTypeVariables #-}  -- 추가 언어 기능 설정

\noindent
문법구조
$\hspace{-2ex}\begin{array}{ll}
& n \in \texttt{Int} \\
& e \in \texttt{Expr} ~::=~ n ~\mid~ e + e
\end{array}$는 데이터 타입으로 선언하면 된다.

In [2]:
data Expr = I Int          -- n
          | Add Expr Expr  -- e + e
       deriving (Eq,Ord,Show) -- Eq 같은지, Ord 순서 비교, Show 출력 가능

작은걸음 의미구조\vspace*{-3ex}
$$
\frac{~ ~}{~ n_1 + n_2 \longmapsto n ~}
{\scriptstyle(\,n\;\equiv\;n_1\,+\,n_2\,)}
\quad
\frac{~ e_1 \longmapsto e_1' ~}{~ e_1 + e_2 \longmapsto e_1' + e_2 ~}
\quad
\frac{~ e_2 \longmapsto e_2' ~}{~ e_1 + e_2 \longmapsto e_1 + e_2' ~}
$$
는 모든 맥락에서 핵심 계산($n_1+n_2\longmapsto n$)을 진행하는 비결정적 의미구조이다.
우선 이 의미구조 그대로를 하스켈 프로그램으로 옮겨 다루어 보고,
그 다음에는 결정적 의미구조로 변형하는 방법에 대해 생각해 보자.

# 작은걸음 의미구조 검산기
\index{작은걸음 의미구조}\index{small-step semantics}

In [3]:
Add (I n1) (I n2) ...> I n  =  n == n1 + n2
Add e1 e2 ...> Add e1' e2   =  e1 ...> e1'  -- 좌변에 변수 e2 중복
Add e1 e2 ...> Add e1  e2'  =  e2 ...> e2'  -- 좌변에 변수 e1 중복
_         ...> _            =  False  -- 다른 경우에는 진행 불가

: 

In [4]:
Add (I n1) (I n2) ...> I n  =  n == n1 + n2
Add e1 e2 ...> Add e1' e2'  =  e1 ...> e1' && e2 == e2'  ||
                               e2 ...> e2' && e1 == e1'
_         ...> _            =  False  -- 다른 경우에는 진행 불가

In [5]:
:type (...>)

(...>) :: Expr -> Expr -> Bool

In [6]:
Add (I 1) (I 2) ...> I 3  -- 검산 성공
I 1 `Add` I 2   ...> I 3
Add (I 1) (I 2) ...> I 4  -- 검산 실패

True

True

False

\noindent 참고로, 하스켈에서 전위(prefix) 표현인 생성자나 함수의 이름을 역따옴표($\,^{\backprime}$)로 감싸면 중위(infix) 연산자처럼 사용할 수 있다.

In [7]:
Add(I 1 `Add` I 2)(I 3 `Add` I 4) ...> Add(I 3)(I 3 `Add` I 4)
Add(I 1 `Add` I 2)(I 3 `Add` I 4) ...> Add(I 1 `Add` I 2)(I 7)
Add(I 1 `Add` I 2)(I 3 `Add` I 4) ...> Add(I 3)(I 7) -- 한꺼번에 2걸음

True

True

False

# 작은걸음 의미구조 계산기
\index{작은걸음 의미구조}\index{small-step semantics}\index{비결정적 의미구조}\index{nondeterministic semantics}
비결정적인 작은걸음 관계($\longmapsto\;\subset\;\texttt{Expr}\times\texttt{Expr}$)를
함수($f_{\longmapsto}$)로 만드는 방법은 모든 가능한 다음 단계를 한꺼번에 모아놓은 집합을
결과값으로 하면 된다. 즉, 식($e\in\texttt{Expr}$) 하나에
식의 집합($\{e_1',e_2',\ldots,e_k'\}\in 2^\texttt{Expr}$) 하나를 대응시키는
함수 $f_{\longmapsto} : \texttt{Expr} \to 2^\texttt{Expr}$를
$f_{\longmapsto}(e) = \{\,e' \mid e\longmapsto e'\,\}$로 정의하면 된다.
이 정의는 단순명료하지만 작은걸음 의미구조의 계산기를 작성하는 데 적합하지 않다.
식 $e$의 문법구조를 따라 정의된 작은걸음 의미구조처럼 $f_{\longmapsto}$를
귀납적(inductive)으로 다음과 같이 정의하면 프로그램으로 옮기기에 좋다.
단, 마지막 등식에서 $e_1+e_2$는 위의 경우들과 겹치지 않는 형태($e_1+e_2\neq n_1+n_2$)라고 가정한다.
\vspace*{-2ex}
\begin{align*}
& f_{\longmapsto} ~ : ~ \texttt{Expr} \to 2^\texttt{Expr} \\
& f_{\longmapsto}(n)             &=~& \{\} \\
& f_{\longmapsto}(n_1 + n_2)     &=~& \{ n \} \qquad(n \equiv n_1 + n_2) \\
& f_{\longmapsto}(\,e_1 + e_2\,) &=~& \{ e_1' + e_2 \mid e_1\longmapsto e_1' \} ~ ~\, \cup ~ ~ \{ e_1 + e_2' \mid e_2\longmapsto e_2'\} \\
&                                &=~& \{ e_1' + e_2 \mid e_1'\in f_{\longmapsto}(e_1) \} \cup \{ e_1 + e_2' \mid e_2'\in f_{\longmapsto}(e_2)\}
\end{align*}
위 정의를 하스켈 리스트로 집합을 표현하는 `step` 함수로 옮기면 다음과 같다.

In [8]:
step :: Expr -> [Expr] -- 리스트로 집합을 표현
step (I n)             = []
step (I n1 `Add` I n2) = [ I n ]  where n = n1 + n2
step (Add e1 e2)       = [ Add e1' e2  | e1' <- step e1 ] ++
                         [ Add e1  e2' | e2' <- step e2 ]

In [9]:
step (I 3)                  -- step 함수 선언의 첫번째 등식에 패턴 매치
step (I 1 `Add` I 2)                       -- 두번째 등식에 패턴 매치 
step (Add (I 1 `Add` I 2) (I 3 `Add` I 4)) -- 세번째 등식에 패턴 매치

[]

[I 3]

[Add (I 3) (Add (I 3) (I 4)),Add (Add (I 1) (I 2)) (I 7)]

## 하스켈 리스트

In [10]:
:type 'a'           -- 문자
:type ['a','b','c'] -- 문자 리스트 = 문자열(String)
:type False         -- 진리값
:type [False, True] -- 진리값 리스트 
:type (&&)          -- 함수
:type [(&&), (||)]  -- 함수 리스트

'a' :: Char

['a','b','c'] :: [Char]

False :: Bool

[False, True] :: [Bool]

(&&) :: Bool -> Bool -> Bool

[(&&), (||)] :: [Bool -> Bool -> Bool]

In [11]:
[True,False,'a','b'] -- 같은 타입끼리만 리스트 구성 가능하므로 오류

: 

### 리스트 이어붙이기 연산자
두 개의 리스트를 다음 연산자로 이어붙일 수 있다.

In [12]:
:type (++)

(++) :: forall a. [a] -> [a] -> [a]

In [13]:
[1,2,3] ++ [4,5,6] -- 정수 리스트 이어붙이기

[1,2,3,4,5,6]

$~$\vspace*{-2ex}\newline
리스트로 집합을 표현한다면 `++` 연산자는 합집합 연산에 해당한다.

참고로, 중위(infix) 연산자만 단독으로 괄호로 감싸면 전위(prefix) 표현인 함수 이름처럼 사용할 수 있다.

In [14]:
(++) [1,2,3] [4,5,6]

[1,2,3,4,5,6]

### 리스트 조건제시법
\index{조건제시법}\index{comprehension}\index{조건제시법!집합 조건제시법}\index{조건제시법!리스트 조건제시법}\index{comprehension!list comprehension}\index{comprehension}{set comprehension} 수학에서 집합 조건제시법(set comprehension)과 비슷한 형태의 문법인 리스트 조건제시법(list comprehension)으로 하스켈 리스트를 작성할 수 있다.\vspace*{-1.5em}

\indent
$\displaystyle\big\{(x,y) \mid x\in\{1,2\},\;y\in\{1,2,3\}\big\}$

In [15]:
[ (x,y) | x <- [1,2], y <-[1,2,3] ]

[(1,1),(1,2),(1,3),(2,1),(2,2),(2,3)]

\indent
$\displaystyle\big\{(x,y,z) \mid x,y,z\in\{1,\ldots,20\},\;x^2+y^2=z^2,\;x<y,\;\text{$x$와 $y$는 서로 소}\big\}$

In [16]:
[ (x,y,z) | x <- [1..20], y <- [1..20], z <- [1..20],
            x^2 + y^2 == z^2, x < y, gcd x y == 1 ]

[(3,4,5),(5,12,13),(8,15,17)]

In [17]:
[1..20]   -- 1부터 1씩 증가하는 20까지의 등차수열
[1,3..20] -- 원소 2개로 시작하면 등차수열 간격 설정

[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

[1,3,5,7,9,11,13,15,17,19]

In [18]:
[ 2^x | x <- [0..10] ] -- 순서쌍 외에 수식, 함수 등으로 계산한 값도 활용 가능

[1,2,4,8,16,32,64,128,256,512,1024]

## 여러 걸음(`step`) 진행하려면

In [19]:
e0 = (Add (I 1 `Add` I 2) (I 3 `Add` I 4))

step e0        -- (1+2)+(3+4)를 1걸음 진행

[Add (I 3) (Add (I 3) (I 4)),Add (Add (I 1) (I 2)) (I 7)]

In [20]:
step (step e0) -- (1+2)+(3+4)를 2걸음 진행???

: 

In [21]:
:type step

step :: Expr -> [Expr]

함수 `step`은 산술식(`Expr`)에 적용해야 하는데 `step`의 결과는 산술식의 리스트(`[Expr]`)이므로
연쇄적으로 적용하려면 타입이 맞지 않는다. $g : A \to A$처럼 정의역과 공역이 같은 함수라야
$g(g(a))$와 같이 연쇄적으로 적용할 수 있다. 일반적으로 $f:A\to 2^A$처럼 하나의 원소를 같은 종류 원소들의 집합으로
대응시키는 함수의 정의역($A$)을 공역($2^A$)에 맞도록 확장한 버전의 함수 $\hat{f}:2^A\to 2^A$를
$\displaystyle\hat{f}(\{a_1,\ldots,a_k\}) = f(a_1)\cup \cdots \cup f(a_k)$, 즉 $\displaystyle\hat{f}(S) = \bigcup_{a\in S} f(a)$로 정의한다.

그렇다면 함수 `step`의 정의역(`Expr`)을 치역(`[Expr]`)에 맞도록 확장한 버전의 함수를 정의해 보자.
우선 첫번째 시도로 아래와 같이 리스트 조건제시법을 이용하면 정의역이 `[Expr]`인 `stepList` 함수를
작성할 수 있기는 한데 치역이 `[[Expr]]`로 2겹 리스트가 되어버린다.

In [22]:
es0 = [e0, I 1 `Add` I 2] -- 산술식 리스트
ess1 = [step e | e <- es0] -- 산술식 리스트의 리스트
:type es0
:type ess1
es0
ess1

es0 :: [Expr]

ess1 :: [[Expr]]

[Add (Add (I 1) (I 2)) (Add (I 3) (I 4)),Add (I 1) (I 2)]

[[Add (I 3) (Add (I 3) (I 4)),Add (Add (I 1) (I 2)) (I 7)],[I 3]]

In [23]:
stepList :: [Expr] -> [[Expr]]
stepList es = [step e | e <- es]

In [24]:
es0
stepList es0

[Add (Add (I 1) (I 2)) (Add (I 3) (I 4)),Add (I 1) (I 2)]

[[Add (I 3) (Add (I 3) (I 4)),Add (Add (I 1) (I 2)) (I 7)],[I 3]]

In [25]:
concat [ [1,2,3], [4,5], [6] ] -- 2겹 리스트를 이어붙여 1겹 리스트로

[1,2,3,4,5,6]

`concat [es1,es2,es3] = es1 ++ es2 ++ es3`와 같이 동작하는 하스켈 표준 라이브러리 함수 `concat :: [[a]] -> [a]`을 이용하면,
`step`의 정의역 `Expr`을 `[Expr]`로 확장한 함수 `step'`을 간단히 정의할 수 있다.

In [26]:
step' :: [Expr] -> [Expr]
step' es = concat [step e | e <- es]

In [27]:
[e0]  -- 식 하나만으로 이루어진 리스트로 시작
step' [e0]                 -- e0에서 1걸음
step' (step' [e0])         -- e0에서 2걸음
step' (step' (step' [e0])) -- e0에서 3걸음

[Add (Add (I 1) (I 2)) (Add (I 3) (I 4))]

[Add (I 3) (Add (I 3) (I 4)),Add (Add (I 1) (I 2)) (I 7)]

[Add (I 3) (I 7),Add (I 3) (I 7)]

[I 10,I 10]

산술식의 계산은 그 과정이 여러갈래로 나누어지더라도 하나의 같은 결과로 수렴하는 수렴성이 있다.
리스트를 내용에 관계없이 그냥 이어붙이는 `++` 연산을 이용하는 `step'`을 연쇄적으로
적용하는 과정에서 `Add (I 3) (I 7)`이나 `I 10`이 중복되어 나타난다.
표준 라이브러리 `Data.List` 모듈로부터 리스트에서 중복된 원소를 제거한 새로운 리스트를 계산하는
함수 `nub`을 불러들여 활용하면 중복 없는 리스트를 계산하도록 `step'`을 다시 정의해 볼 수도 있다.

In [28]:
import Data.List (nub)
nub [1,2,3,2,4,3,5,1]
nub [I 10, I 10]

[1,2,3,4,5]

[I 10]

In [29]:
step' :: [Expr] -> [Expr]
step' es = nub (concat [step e | e <- es])

In [30]:
[e0] -- 식 하나만으로 이루어진 리스트로 시작
step' it   -- e0에서 1걸음 (it은 직전 실행한 식의 결과값)
step' it   -- e0에서 2걸음 (it은 직전 실행한 식의 결과값)
step' it   -- e0에서 3걸음 (it은 직전 실행한 식의 결과값)

[Add (Add (I 1) (I 2)) (Add (I 3) (I 4))]

[Add (I 3) (Add (I 3) (I 4)),Add (Add (I 1) (I 2)) (I 7)]

[Add (I 3) (I 7)]

[I 10]

## 고차함수(higher-order function)
\index{고차함수|see{higher-order function}}\index{higher-order function|see{고차함수}} `step`을 확장한 `step'`은 일반적으로 함수 $f:A\to 2^A$를 $\hat{f}:2^A\to 2^A$로 확장하는
특별한 사례일 뿐이다. 그렇다면 일반적으로 함수에 적용해 확장된 함수를 계산하는
$\hat{\cdot}:(A\to 2^A)\to(2^A\to 2^A)$ 연산 자체를 하스켈 프로그램으로 표현할 수는 없을까?
하스켈과 같은 함수형 언어에서는 너무나 당연하고 자연스럽게 아래와 같이 가능하다.

In [31]:
hat f = \es -> nub (concat [f e | e <- es])

In [32]:
step' = hat step  -- hat으로 step' 선언
:type step'
[e0]  -- 식 하나만으로 이루어진 리스트로 시작
step' [e0]                 -- e0에서 1걸음
step' (step' [e0])         -- e0에서 2걸음
step' (step' (step' [e0])) -- e0에서 3걸음

step' :: [Expr] -> [Expr]

[Add (Add (I 1) (I 2)) (Add (I 3) (I 4))]

[Add (I 3) (Add (I 3) (I 4)),Add (Add (I 1) (I 2)) (I 7)]

[Add (I 3) (I 7)]

[I 10]

하스켈과 같은 함수형 언어에서는 `hat`처럼 함수를 인자로 받는 함수를 고차함수라고 한다.
참고로, 커리된 함수를 자연스럽게 다루지 못하는 프로그래밍언어에서는 함수를 인자로 받지 않더라도
함수의 결과값이 함수이면 고차함수라고 부르기도 한다.
하스켈처럼 커리하기(currying)를 잘 지원하고 장려하는 언어에서는
대부분 함수를 커리된 함수로 정의하므로 함수가 결과값인 함수는
너무나 보편적이라 따로 분류해 생각할 필요조차 없다.

중고교 수학과정을 통해 친숙한 대표적인 고차함수는 바로
함수 합성 연산자 $\cdot\circ\cdot : (B\to C) \to (A\to B) \to (A\to C)$로,
$g:B\to C$와 $f:A\to B$의 합성함수 $h = g\circ f$는 $h(x) = g(f(x))$와 같이 동작하는 함수다.
하스켈에 표준 라이브러리에서는 함수 합성 연산자 `(.) :: (b -> c) -> (a -> b) -> (a -> c)`를 제공한다.

In [33]:
:type (.)

(.) :: forall b c a. (b -> c) -> (a -> b) -> a -> c

In [34]:
:type \(x :: Int) -> [x, x+1, x+2]
:type \(xs::[Int]) -> [(x,x^2) | x <- xs]

\(x :: Int) -> [x, x+1, x+2] :: Int -> [Int]

\(xs::[Int]) -> [(x,x^2) | x <- xs] :: [Int] -> [(Int, Int)]

In [35]:
f1 = \(x :: Int) -> [x, x+1, x+2]
g1 = \(xs::[Int]) -> [(x,x^2) | x <- xs]
h1 = g1 . f1
:type h1

h1 :: Int -> [(Int, Int)]

In [36]:
f1 3
g1(f1 3)
h1 3

[3,4,5]

[(3,9),(4,16),(5,25)]

[(3,9),(4,16),(5,25)]

\noindent
논리부정 `not :: Bool -> Bool`과
빈 리스트인지 검사하는 `null :: [a] -> Bool`의 합성
`(not . null) :: [a] -> Bool`은 채워진 리스트인지 검사한다.

In [37]:
null [1,2]
null []

False

True

In [38]:
(not . null) [1,2]
(not . null) []

True

False

### 리스트를 다루는 고차함수
고차함수가 아닌 표준 라이브러리 함수 `take, drop :: Int -> [a] -> [a]`는
특정 개수만큼의 원소로 이루어진 앞부분과 그 앞부분을 제외한 나머지 리스트를
아래와 같이 계산한다. 참고로 하스켈은 게으른 계산법을 따르므로
무한 리스트를 다루는 계산도 표현 가능하며, 무한 리스트로부터
필요로 하는 만큼의 유한한 내용만 최종적인 결과로 유도하는
실용적인 계산 과정에 활용할 수 있다.

In [39]:
take 3 [1,3,5,7,9] -- 유한한 리스트의 앞부분 3개
drop 3 [1,3,5,7,9] -- 앞부분 3개를 제외한 나머지
take 3 [1..]       -- 무한한 리스트의 앞부분 3개
-- drop 3 [1..]    -- 이걸 실행하면 무한 리스트 [4..]

[1,3,5]

[7,9]

[1,2,3]

\noindent
고차함수 `takeWhile, dropWhile :: (a -> Bool) -> [a] -> [a]`는 개수 대신
특정 조건을 검사하는 함수를 인자로 받아 그 조건이 유지되는 앞부분과
그 앞부분을 제외한 나머지 리스트를 아래와 같이 계산한다.

In [40]:
takeWhile (\x -> x<6) [1,3,5,7,9] -- 유한 리스트의 6미만으로 유지되는 앞부분
dropWhile (\x -> x<6) [1,3,5,7,9] -- 6미만이 유지되는 앞부분을 제외한 나머지
takeWhile (\x -> x<6) [1..]       -- 무한 리스트의 6미만으로 유지되는 앞부분
-- dropWhile (\x -> x<6) [1..]    -- 이걸 실행하면 무한 리스트 [7..]

[1,3,5]

[7,9]

[1,2,3,4,5]

고차함수 `iterate :: (a -> a) -> a -> [a]`는 주어진 함수를 특정 초기값에 반복해서
연쇄적으로 적용한 각 단계의 계산 결과를 나열하는 무한 리스트를 만들어낸다. 즉,
`iterate f x = [x, f x, f(f x), f(f(f x)), ... ]`처럼 동작한다.
예컨대, 1로 시작하는 등비 2의 무한 수열을 `iterate (\x -> x*2) 1`로 나타낼 수 있다.
그 중 앞부분 12개만 취해 보면 다음과 같다.

In [41]:
take 12 (iterate (\x -> x*2) 1)

[1,2,4,8,16,32,64,128,256,512,1024,2048]

확장된 작은걸음 함수 `step'`에 `iterate`를 적용하여 각 단계의 작은걸음 계산 결과를 나열하는 무한 리스트를 만들어낼 수 있다. 적절한 개수의 앞부분을 `take`로 취해 보면 계산이 완료되어 더 이상 진행할 수 없는 단계부터는 빈 리스트가 계속 나타남을 확인할 수 있다. 따라서 비어있지 않은 채워진 리스트라는 조건(`not . null`)이 유지되는 앞부분만 `takeWhile`로 취하면 산술식의 작은걸음 계산이 완료되는 단계까지만을 나열한 산술식 리스트의 리스트를 얻을 수 있다.

In [42]:
take 10                (iterate step' [e0])
takeWhile (not . null) (iterate step' [e0])

[[Add (Add (I 1) (I 2)) (Add (I 3) (I 4))],[Add (I 3) (Add (I 3) (I 4)),Add (Add (I 1) (I 2)) (I 7)],[Add (I 3) (I 7)],[I 10],[],[],[],[],[],[]]

[[Add (Add (I 1) (I 2)) (Add (I 3) (I 4))],[Add (I 3) (Add (I 3) (I 4)),Add (Add (I 1) (I 2)) (I 7)],[Add (I 3) (I 7)],[I 10]]

In [43]:
mapM_ print [1,2,3] -- 리스트의 각 원소를 한줄씩 출력해 보려면 이렇게

1
2
3

In [44]:
mapM_ print (takeWhile (not . null) (iterate step' [e0]))

[Add (Add (I 1) (I 2)) (Add (I 3) (I 4))]
[Add (I 3) (Add (I 3) (I 4)),Add (Add (I 1) (I 2)) (I 7)]
[Add (I 3) (I 7)]
[I 10]

# 결정적 작은걸음 의미구조
\index{작은걸음 의미구조}\index{small-step semantics}\index{결정적 의미구조}\index{deterministic semantics} 항상 왼쪽을 우선하며 핵심적인 계산($n_1+n_2\xmapsto{~_L~} n$)을 진행하는
산술식의 결정적 작은걸음 의미구조($\xmapsto{~_L~}$)를 다음과 같이 정의할 수 있다.
$$
\frac{~ ~}{~ n_1 + n_2 \xmapsto{~_L~} n ~}
{\scriptstyle(\,n\;\equiv\;n_1\,+\,n_2\,)}
\quad
\frac{~ e_1 \xmapsto{~_L~} e_1' ~}{~ e_1 + e_2 \xmapsto{~_L~} e_1' + e_2 ~}
\quad
\frac{~ e_2 \xmapsto{~_L~} e_2' ~}{~ n_1 + e_2 \xmapsto{~_L~} n_1 + e_2' ~}
$$
이 장 앞부분에 소개했던 비결정적 의미구조 $\longmapsto$과 차이점은
마지막 규칙에서 덧셈식의 왼항이 계산이 완료된 정수값 형태($n_1$)일 때만
오른항($e_2$)의 계산을 진행하도록 제한한다는 점이다. 위의 의미구조
$\xmapsto{~_L~}$은 모든 산술식에 대해 단 하나의 규칙만이 대응된다.
덧셈식의 양쪽 항이 모두 정수값 형태인 경우는 첫째 규칙만,
오른항의 계산이 완료되지 않은 형태의 식이면 둘째 규칙만,
오른항이 정수값 형태이고 왼항이 계산이 완료되지 않은 형태의 식이면
마지막 셋째 규칙만 적용 가능하다.

In [45]:
-- 제대로 동작하지 않는 검산기 구현 시도
Add (I n1) (I n2) /..> I n          = n == n1 + n2             -- 첫째
Add e1     e2 /..> Add e1'     e2'  = e1 /..> e1' && e2 == e2' -- 둘째
Add (I n1) e2 /..> Add (I n1') e2'  = e2 /..> e2' && n1 == n1' -- 셋째
_             /..> _                = False  -- 다른 경우에는 진행 불가

\noindent
작은걸음 규칙을 차례대로 하스켈 연산자 `/..>`로 선언하여 위와 같은 옮겨보면
거의 올바른 듯이 보이지만 셋째 규칙을 구현한 등식이 제대로 동작하지 않는다.

In [46]:
Add (I 1) (I 2) /..> I 3 -- 검산 성공
Add (I 1 `Add` I 2) (I 7) /..> Add (I 3) (I 7) -- 검산 성공
Add (I 7) (I 1 `Add` I 2) /..> Add (I 7) (I 3) -- 검산 성공해야 하는데??

True

True

False

\noindent
왜냐하면 `/..>`의 선언에서 둘째 등식의 왼쪽 매개변수 패턴 `Add e1 e2`가
셋째 등식의 왼쪽 매개변수 패턴 `Add (I n1) e2`보다 일반적이라서,
위에서부터 아래로 가장 먼저 맞춰지는 패턴에 따라 실행되기 때문에
셋째 등식의 패턴에 맞춰질만한 인자는 모두 둘째 등식에서 이미 맞춰지므로
둘째 등식이 실행되고 셋째 등식은 실행되는 경우가 없게 된다. 추론규칙의
형태로 정의한 의미구조는 규칙이 배치된 순서와 무관하게 가능한 규칙을
모두 적용하지만, 하스켈 프로그램은 위에서부터 아래 순서로 가장 먼저
매칭되는 패턴에 해당하는 내용만 실행한다는 점에 유의해야 한다. 따라서
아래와 같이 다시 더 구체적인 패턴을 다루는 둘째 규칙에 대한 등식이
셋째 등식에 대한 규칙보다 앞서 나타나도록 정의해야 의미구조 $\xmapsto{~_L~}$를
추론규칙으로 정의한 의도한 대로 동작하게 된다.

In [47]:
Add (I n1) (I n2) /..> I n          = n == n1 + n2             -- 첫째
Add (I n1) e2 /..> Add (I n1') e2'  = e2 /..> e2' && n1 == n1' -- 셋째
Add e1     e2 /..> Add e1'     e2'  = e1 /..> e1' && e2 == e2' -- 둘째

In [48]:
Add (I 1) (I 2) /..> I 3 -- 검산 성공
Add (I 1 `Add` I 2) (I 7) /..> Add (I 3) (I 7) -- 검산 성공
Add (I 7) (I 1 `Add` I 2) /..> Add (I 7) (I 3) -- 검산 성공!!

True

True

True

In [49]:
stepL :: Expr -> [Expr]
stepL (I n1) = []
stepL (I n1 `Add` I n2) = undefined -- 계산기를
stepL (I n1 `Add` e2)   = undefined -- 완성하는
stepL (e1   `Add` e2)   = undefined -- 연습문제

In [50]:
-- 완성한 stepL을 적절히 테스트해 보라

In [51]:
stepL' :: [Expr] -> [Expr]  -- 확장된 작은걸음 함수를
stepL' es = undefined       -- 작성하는 연습문제 

In [52]:
-- 작성한 stepL'를 적절히 테스트해 보라

# 값계산 맥락(evaluation context)
\index{값계산 맥락|see{evaluation context}}\index{evaluation context|see{값계산 맥락}} 문법구조의 각 요소마다 계산의 핵심 규칙을 어느 맥락에 적용할지 규정하는
맥락규칙을 개별적인 추론규칙으로 작성하는 방식은 자리를 많이 차지하므로
지면을 낭비하여 다양한 요소를 갖춘 문법구조를 다룰수록 한눈에 알아보기
어려운 측면이 있다. 이 장에서 예시로 다루는 정수 리터럴과 덧셈만으로
이루어진 산술식(`Expr`)은 간단한 예시를 만들고자 최소한의 요소만으로
이루어져 있으므로 핵심 규칙 하나 외에 맥락 규칙은 두 개밖에 없다.
하지만 여기에 순서쌍, 조건식 등 다양한 복합적 문법요소가 추가된다면
각 요소마다 여러개의 맥락 규칙이 함께 추가되어야 한다.

이렇게 핵심 규칙에 비해 맥락 규칙의 개수가 훨씬 많아지는
의미구조 정의의 번잡함을 줄이고자, 맥락(context)이라는 개념을
명시적으로 드러내 활용하며 개별적으로 작성하던 맥락 규칙들을
하나의 맥락규칙으로 통합하여 나타내는 아래와 같은 작은걸음 의미구조의
표현방식을
\index{맥락적 의미구조|see{contextual semantics}}\index{contextual semantics|see{맥락적 의미구조}}
\`맥락적 의미구조'(contextual semantics)라 일컫는다.
특히 계산식을 실행한 값을 다루는 의미구조에서
이러한 맥락 $\mathcal{E}$를 \`값계산 맥락'이라고도 부른다.
\vspace*{-2ex}
$$
\frac{~ ~}{~ n_1 + n_2 \xmapsto{~_B~} n ~}
{\scriptstyle(\,n\;\equiv\;n_1\,+\,n_2\,)}
\qquad\qquad
\frac{~ e \xmapsto{~_B~} e' ~}{~ \mathcal{E}[\,e\,] \xmapsto{~_C~} \mathcal{E}[\,e'] ~}
$$
맥락적 의미구조($\xmapsto{~_C~}$)에서는 맥락 $\mathcal{E}$를 어떻게 정의하느냐에
따라 조금씩 다른 계산 우선순위를 따르는 의미구조가 정의된다.
값계산 맥락을 \ref{evctx:nd}로 정의하면 이 장의 첫머리에 소개한
비결정적 의미구조($\longmapsto$)와 일치하며 \ref{evctx:l}로 정의하면
왼쪽부터 계산하는 결정적 의미구조($\xmapsto{~_L~}$)와 일치하게 된다.
\vspace*{-1ex}
\begin{align}
\mathcal{E} ::=\;& \bullet \mid \mathcal{E}+e \mid e+\mathcal{E} 
\label{evctx:nd} \\
\mathcal{E} ::=\;& \bullet \mid \mathcal{E}+e \mid n+\mathcal{E}
\label{evctx:l}
\end{align}
맥락 $\mathcal{E}$를 직관적으로 설명하자면 산술식의 어떤 부분에
원래 그 부분을 차지하고 있던 부분식 대신 구멍($\bullet$)을 뚫어놓아
필요하다면 원래 부분식과 다른 부분식을 그 차리에 채워넣을 수 있도록
만들어 놓은 것으로 이해하면 된다. 그리고 위의 \ref{evctx:nd}이나
\ref{evctx:l}같은 $\mathcal{E}$의 정의는 어느 부분에 구멍을 뚫어도
되는지 허용하는 귀납적인 규칙이다. $\bullet$은 식 전체에,
$\mathcal{E}+e$는 덧셈식 왼쪽에 어딘가에,
$e+\mathcal{E}$는 덧셈식 오른쪽 어딘가에
(귀납적 규칙에 따라) 구멍을 뚫을 수 있음을 나타내며,
$n+\mathcal{E}$은 덧셈식 왼쪽이 정수값인 경우에만 오른쪽 어딘가에
(귀납적 규칙에 따라) 구멍을 뚫을 수 있음을 나타낸다.
맥락에는 항상 1개의 구멍만 뚫려 있어야야 하므로
$\mathcal{E}+\mathcal{E}$처럼 구멍이 두 군데 뚫려 있다고 나타내는
식의 맥락 정의는 허용되지 않는다. 맥락 규칙에서 맥락을 활용하는 표현
$\mathcal{E}[e]$는 맥락 $\mathcal{E}$의 구멍 뚫린 부분에 $e$를
채워넣어 만들어진 구멍 없는 온전한 산술식을 나타낸다. 예컨대,
$\mathcal{E}=4 + (3 + \bullet)$이고 $e=2+1$이라면
$\mathcal{E}[e] = (4 + (3 + \bullet))[2+1] = 4+(3+(2+1))$이다.

맥락을 하스켈 데이터 타입으로 정의할 수도 있지만\cite{Hutton2021easy123}
여기서는 식을 인자로 받아 새로운 식을 계산하는 함수로 정의하겠다.
방금 위에서 살펴본 예에서도 $4+(3+\bullet)$이라는 맥락은
식 $2+1$를 받아서 새로운 식 $4+(3+(2+1))$를 만들어내도록 활용되었다.
따라서 $4+(3+\bullet)$를 하스켈 함수로 다음과 같이 옮겨 `ctx1`으로 선언하고
활용할 수 있다.

In [53]:
ctx1 = \e -> Add (I 4) (I 3 `Add` e) 
:type ctx1
ctx1 (I 2 `Add` I 1)
ctx1 (I 99)

ctx1 :: Expr -> Expr

Add (I 4) (Add (I 3) (Add (I 2) (I 1)))

Add (I 4) (Add (I 3) (I 99))

맥락적 의미구조에서는 맥락 규칙이 적용 가능한 모든 맥락을 다 고려해야 한다.
여기서는 비결정적 의미구조에 해당하는 \ref{evctx:nd}의 맥락 정의를 따르자.
즉, 맥락 규칙으로 한걸음 $e_0\xmapsto{~_C~}e_1$을 진행하려면
주어진 식 $e_0$에 대해 $e_0 = \mathcal{E}[e]$인 가능한 모든 맥락
$\mathcal{E}$와 구멍뚫린 위치에 있는 부분식 $e$를 생성하여,
그 중에서 전제 조건 $e\xmapsto{~_B~}e'$에 맞는 맥락에만 $e'$를 채워넣은
$e_1 = \mathcal{E}[e']$로 진행 가능하다.

\ref{evctx:nd}의 맥락 정의에 따라, 주어진 식에 대한 맥락과
구멍뚫린 곳의 부분식으로 이루어진 가능한 모든 순서쌍 $(\mathcal{E},e)$를
생성하는 하스켈 함수 `split :: Expr -> [(Ctx,Expr)]`를 다음과 같이 선언할 수 있다.

In [54]:
type Ctx = Expr -> Expr -- 타입 별명으로 긴 타입을 한 단어로 줄여 표시
split :: Expr -> [(Ctx,Expr)]
split (I n)       = [(\e-> e, I n)]
split (Add e1 e2) = [(\e-> e, Add e1 e2)]
                 ++ [(\e-> Add (ctx e) e2, e) | (ctx,e) <- split e1]
                 ++ [(\e-> Add e1 (ctx e), e) | (ctx,e) <- split e2]

In [55]:
-- (1+2)+(3+4)에 구멍을 뚫는 모든 경우를 나열한 리스트
split (Add (I 1 `Add` I 2) (I 3 `Add `I 4))

: 

\noindent
맥락의 타입에 해당하는 `Ctx`는 함수 타입인 `Expr -> Expr`의 타입 별명이다.
그런데 하스켈에서 함수는 기본적으로 문자열로의 변환을 통해 출력되는 부류의 타입이 아니다.
그러니까 맥락을 알아볼 수 있도록 출력하기 위해
`Ctx`에 대한 타입 클래스 `Show`의 인스턴스를 직접 제공하여
출력되는 부류의 타입으로 등록한 다음 `split`으로 계산한 결과값을 다시 확인해 보자.

In [56]:
{-# LANGUAGE FlexibleInstances #-}
instance Show Ctx where
  show ctx = show (ctx (I minBound)) -- 구멍 위치에 정수 리터럴 최소값 표시

In [57]:
split (Add (I 1 `Add` I 2) (I 3 `Add `I 4))

[(I (-9223372036854775808),Add (Add (I 1) (I 2)) (Add (I 3) (I 4))),(Add (I (-9223372036854775808)) (Add (I 3) (I 4)),Add (I 1) (I 2)),(Add (Add (I (-9223372036854775808)) (I 2)) (Add (I 3) (I 4)),I 1),(Add (Add (I 1) (I (-9223372036854775808))) (Add (I 3) (I 4)),I 2),(Add (Add (I 1) (I 2)) (I (-9223372036854775808)),Add (I 3) (I 4)),(Add (Add (I 1) (I 2)) (Add (I (-9223372036854775808)) (I 4)),I 3),(Add (Add (I 1) (I 2)) (Add (I 3) (I (-9223372036854775808))),I 4)]

In [58]:
mapM_ print it -- 직전 결과값 리스트의 원소를 한줄에 하나씩 출력

(I (-9223372036854775808),Add (Add (I 1) (I 2)) (Add (I 3) (I 4)))
(Add (I (-9223372036854775808)) (Add (I 3) (I 4)),Add (I 1) (I 2))
(Add (Add (I (-9223372036854775808)) (I 2)) (Add (I 3) (I 4)),I 1)
(Add (Add (I 1) (I (-9223372036854775808))) (Add (I 3) (I 4)),I 2)
(Add (Add (I 1) (I 2)) (I (-9223372036854775808)),Add (I 3) (I 4))
(Add (Add (I 1) (I 2)) (Add (I (-9223372036854775808)) (I 4)),I 3)
(Add (Add (I 1) (I 2)) (Add (I 3) (I (-9223372036854775808))),I 4)

기본적인 핵심 계산 규칙($\xmapsto{~_B~}$)과
하나로 통합된 맥락 규칙($\xmapsto{~_C~}$)을
구현하는 계산기 함수 `stepB`와 `stepC`를
다음과 같이 `split`을 활용해 선언할 수 있다.

In [59]:
stepB :: Expr -> [Expr]
stepB (I n1 `Add` I n2) = [I n] where n = n1 + n2
stepB _                 = []
stepC :: Expr -> [Expr]
stepC e0 = [ctx e' | (ctx,e) <- split e0, e' <- stepB e]

비결정적 의미구조를 실행해 보았을 때와 같은 산술식(`e0`)에
`stepC`를 확장한 `hat stepC`를 연쇄적으로 적용하여 여러 단계를
진행한 단계별 결과를 다음과 같이 확인해 보면 지금까지 우리가
구현한 맥락적 의미구조(`stepC`)가 비결정적 의미구조(`step`)와
똑같이 계산이 진행됨을 확인해 볼 수 있다.

In [60]:
mapM_ print ( takeWhile (not . null) (iterate (hat stepC) [e0]) )

[Add (Add (I 1) (I 2)) (Add (I 3) (I 4))]
[Add (I 3) (Add (I 3) (I 4)),Add (Add (I 1) (I 2)) (I 7)]
[Add (I 3) (I 7)]
[I 10]

# 큰걸음 의미구조
\label{sec:bigStepArith}
\index{큰걸음 의미구조}\index{big-step semantics}
계산의 진행 과정보다는 계산의 결과값에만 주로 관심이 있을 때
적합한 동작과정 의미구조가 바로 큰걸음 의미구조이다. 지금까지
작은걸음 의미구조로 다루던 산술식에 나눗셈 연산을 추가한
산술식 $e\in\texttt{Expr} \;::=\; n \;\mid\; e+e \;\mid\; e\div e$의 큰걸음 의미구조는 다음과 같이 표현할 수 있다.
\qquad
$\displaystyle\frac{~ ~}{~ n \Longmapsto n ~}$
$$
\frac{~ e_1 \Longmapsto n_1
  \quad e_2 \Longmapsto n_2 ~}{~ 
       e_1 + e_2 \Longmapsto n ~}
({\scriptstyle n\;\equiv\;n_1+n_2})
\quad~~
\frac{~ e_1 \Longmapsto n_1
  \quad e_2 \Longmapsto n_2 ~}{~ 
       e_1 \div e_2 \Longmapsto n ~}
({\scriptstyle n_2\;\neq\;0,~n\;\equiv\;n_1 \div n_2})
$$
이를 하스켈 프로그램으로 옮기면 다음과 같다.

In [61]:
data Expr = I Int | Add Expr Expr | Div Expr Expr 
          deriving (Eq,Ord,Show)

In [62]:
bigStep :: Expr -> [Int]
bigStep (I n)       = [ n ]
bigStep (Add e1 e2) = [ n1   +   n2 | n1 <- bigStep e1,
                                      n2 <- bigStep e2 ]
bigStep (Div e1 e2) = [ n1 `div` n2 | n1 <- bigStep e1,
                                      n2 <- bigStep e2, n2 /= 0 ]

In [63]:
bigStep (I 3)
bigStep (Add (I 1 `Add` I 2) (I 3 `Add` I 4))
bigStep (I 10 `Div` I 4)
bigStep (I 10 `Div` I 0) -- 규칙에 따라 값을 구할 수 없으면 빈 리스트

[3]

[10]

[2]

[]

큰걸음 의미구조는 결과값만을 중시하므로 합류성(confluence)이
만족하는 경우라면 계산 과정에서의 연산 우선순위에 크게 관계없이
같은 결과값에 도달하므로 여러 가지 작은걸음 의미구조에 대응될 수 있다.

참고로, 정수 리터럴과 몇 가지의 연산으로만 이루어진 산술식과
같이 단순한 언어의 경우에는 큰걸음 의미구조의 정의와
지시적 의미구조의 정의가 거의 비슷한 구조를 갖는다.
다음의 지시적 의미구조에서, $+_{\textsc{m}}$와 $\div_{\textsc{m}}$는
양쪽 피연산자가 모두 정수인 경우에는 정수 덧셈과 나눗셈을 하지만
두 피연산자 중 하나라도 오류를 의미하는 $\bot$인 경우에는 연산의 결과도 $\bot$이다.
추가로 $\div_{\textsc{m}}$에서는 양쪽 피연산자가 모두 정수라도
나누는 수인 오른쪽 피연산자가 0인 경우에는 연산의 결과가 $\bot$이 된다.
\vspace*{-4ex}
\begingroup\addtolength{\jot}{-1ex}
\begin{align*}
\llbracket\,\cdot\,\rrbracket &\;~:~\; \texttt{Expr} \to \{\bot\}\cup\mathbb{Z} \\
\llbracket\,n\,\rrbracket & ~=~ n \\
\llbracket\,e_1 +\, e_2\,\rrbracket & ~=~
  \llbracket e_1\rrbracket \;+_{\textsc{m}}\; \llbracket e_2\rrbracket \\
\llbracket\,e_1 \div e_2\,\rrbracket & ~=~ 
  \llbracket e_1\rrbracket \,\div_{\textsc{m}}\, \llbracket e_2\rrbracket
\end{align*}
\vspace*{-5ex}\newline
\endgroup
이를 하스켈 프로그램으로 옮기면 아래와 같다.
`Maybe Int` 타입으로 $\{\bot\}\cup\mathbb{Z}$을 나타내며,
이 타입의 값들인 `Nothing`은 $\bot$를 `Just n`는 그냥 정수 $n$을 나타낸다.

In [64]:
eval :: Expr -> Maybe Int
eval (I n) = Just n 
eval (Add e1 e2) = eval e1 `addM` eval e2
eval (Div e1 e2) = eval e1 `divM` eval e2

addM :: Maybe Int -> Maybe Int -> Maybe Int
Just n1 `addM` Just n2 = Just (n1 + n2)
_       `addM` _       = Nothing
divM :: Maybe Int -> Maybe Int -> Maybe Int
Just n1 `divM` Just 0  = Nothing
Just n1 `divM` Just n2 = Just (n1 `div` n2)
_       `divM` _       = Nothing

In [65]:
eval (I 3)
eval (Add (I 1 `Add` I 2) (I 3 `Add` I 4))
eval (I 10 `Div` I 4)
eval (I 10 `Div` I 0) -- 0으로 나누면 오류

Just 3

Just 10

Just 2

Nothing

하스켈에서 `Maybe a` 타입은 보통의 정상적인 결과값은 `a`타입으로 계산되지만
정상적인 값에 대응되지 않는 경우도 있는 함수의 공역을 나타내는 용도로 쓰이며,
표준 라이브러리에 다음과 같은 데이터 타입으로 선언되어 있다.
```haskell
      data Maybe a = Nothing | Just a   deriving (Eq, Ord, ...... )
```
위의 데이터 타입 선언은 하나의 구체적인 타입을 정의하는 것이 아니라
타입 매개변수(type parameter) `a`가 어떤 타입으로 구체화(instantiate)되느냐에
따라 여러 가지 다른 타입으로 구체화된다. 위의 `eval` 코드에서는 `Maybe`의
타입 매개변수를 `Int`로 구체한 `Maybe Int`라는 타입을 활용하며, 그에
따라 생성자 `Just`의 인자로 `Int` 타입의 정수값이 온다.
즉, `n :: Int`일 때 `Just n :: Maybe Int`이다. 한편, `Nothing`은
`a`와 무관하게 정의되므로 타입 매개변수 `a`가 어떻게 구체화되든
관계없이 항상 사용할 수 있다. 이렇게 `Maybe`와 같이 타입 매개변수를
활용해 선언된 타입을 \`매개변수화된 타입'(parameterized type)이라 일컫는다.

$~$
\section*{요점정리}
* 작은걸음 의미구조에서 한 걸음 $e \longmapsto e'$이 의미구조를 정의하는 규칙에 맞는지
  판별하는 검산기는 `Expr -> Expr -> Bool` 타입의 이항 연산으로 작성할 수 있다.
* 작은걸음 의미구조의 규칙에 따라 주어진 식으로부터 한 걸음 다음의 산술식을 구하는
  계산기는 `Expr -> [Expr]` 타입의 하스켈 함수로 작성할 수 있다.
  함수의 결과값이 리스트인 이유는
  더 이상 진행할 수 없는 값은 한 걸음 다음의 식이 0개이며
  비결정적 의미구조인 경우에는 한 걸음 다음으로 가능한 식이 여럿일 수도 있기 때문이다.
* `Expr -> [Expr]` 타입의 작은걸음 계산기는 연쇄적으로 적용하기 불편하므로
   정의역을 치역과 일치시킨 `[Expr] -> [Expr]` 타입의 확장된 함수를 선언하여 활용하면
   여러 작은걸음 이후의 식을 계산하기에 좋다.
* 하스켈처럼 커리하기(currying)를 잘 지원하는 언어에서 고차함수는
  함수를 인자로 받는 함수를 말한다. 커리된 함수를 활용하기 번거로운
  언어에서는 함수를 결과값으로 계산하기만 해도 고차함수라 부르기도 한다.
* 리스트를 다루는 고차함수를 활용하면 `[Expr] -> [Expr]` 타입의 확장된 산술식
  작은걸음 계산기를 적용한 계산의 각 진행 단계에 나타나는 산술식을 한 줄의
  하스켈 프로그램으로 간단히 확인해볼 수 있다.
* 결정적 의미구조는 하나의 식에 적용할 수 있는 의미구조 규칙이
  최대 한개씩만 있도록 정의된다.
* 비결정적 의미구조를 결정적 의미구조로 변경하는 방법은
  일부 규칙이 더 제한된 형태의 식에만으로 적용되도록 그 적용 범위를 좁히도록
  변경함으로써 다른 규칙들과 적용 범위가 겹치지 않도록 하는 것이다.
* 맥락적 의미구조는 기본적 핵심 계산 규칙 이외에 핵심 계산을 적용하는 맥락을
  규정하는 규칙들을 하나의 맥락 규칙으로 통합하여 정의하는 의미구조이다.
  이렇게 통합된 맥락 규칙은 식에 구멍이 하나 뚫려 있는 구조인
  값계산 맥락(evaluation context)을 이용해 정의된다.
* 값계산 맥락을 어떻게 정의하느냐에 따라 맥락적 의미구조가
  비결정적 의미구조와 일치할 수도 있고
  특정 맥락의 계산을 우선하는 결정적 의미구조와 일치할 수도 있다.
* 큰걸음 의미구조는 계산 과정보다는 계산의 결과값만을 주로 다루기에
  적합한 동작과정 의미구조의 정의 방식이다. 그렇기에 큰걸음 의미구조는
  계산의 결과는 같지만 계산 과정에서 우선 순위가 다른 여러 가지 작은걸음
  의미구조에 대응될 수 있다.

\section*{연습문제}
1. 작은걸음 계산기 `step`이 규칙에 맞게 구현되었는지 검산기(`...>`)로 확인해 보라.
   `step`을 적용한 인자와 계산된 결과값 리스트의 원소를 반복해서
   검산기로 확인하는 하스켈 코드를 여러 개의 식으로 작성하는 방법 말고,
   인자를 리스트의 모든 원소에 대해 하나의 식으로 한꺼번에 검산하는 방법을 생각해 보라.
1. 확장된 작은걸음 계산기 `step'`이 규칙에 맞게 구현되었는지 검산기(`...>`)로
   확인해 보라. `step'`을 적용한 인자 리스트의 원소와 계산된 결과 리스트의 원소를
   반복해서 연관지어 가며 검산기로 확인하는 하스켈 코드을 여러 개의 식으로 작성하는
   방법 말고, 인자와 결과값 리스트의 모든 원소에 대해 하나의 식으로 한꺼번에
   검산하는 방법을 생각해 보라.
1. 왼쪽 맥락부터 핵심 계산을 진행하는 결정적 작은걸음 의미구조 계산기 `stepL`의
   선언를 완성하고 이를 연쇄적으로 적용하기 편리하게 확장한 함수 `stepL'`의 선언도
   작성해 보라. 또 제대로 이 두 함수가 제대로 선언되었는지 확인하는 적절한 테스트를
   각각의 함수에 대해 수행하라.
1. 오른쪽 맥락부터 핵심 계산을 진행하는 결정적 작은걸음 의미구조 $\xmapsto{~_R~}$을
   정의해 보라. 또한 $\xmapsto{~_R~}$과 일치하는 맥락적 의미구조($\xmapsto{~_C~}$)가 되도록
   값계산 맥락 $\mathcal{E}$를 정의해 보라.
1. 위 문제의 맥락 정의에 맞게 `[45]`번 셀의 `split` 함수를 수정하고
   `[60]`번 셀까지 실행 결과가 어떻게 바뀌는지 확인해 보라.
1. 나눗셈까지 포함된 산술식의 큰걸음 의미구조에 대응되는
   맥락적 의미구조를 정의해 보라. 비결정적인 의미구조로 정의해 보아도
   좋고 결정적인 의미구조로 정의해 보아도 좋다.

\section*{탐구과제}
1. 하스켈 표준 라이브러리에는 `map`, `filter`, `foldr`, `foldl` 등
   이 장에서 소개하지 않은 리스트를 다루는 고차함수들이 더 많이 있다.
   이들에 대해 조사해 보라.
1. 동작과정 의미구조와 지시적 의미구조, 증명 이론과 모델 이론은
   어떤 관련이 있는지 알아보라. 